In [4]:
!ngrok config add-authtoken 2PymkjIsHZKpTmllxZ9EuS9now4_T8QSVRfG4Ko5ZhzwgZdC

!pip install fastapi uvicorn transformers torch pyngrok

from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline
from pyngrok import ngrok
import uvicorn
import nest_asyncio


# Permitir el uso de asyncio en Colab
nest_asyncio.apply()

# crear instancia de fastApi
app = FastAPI()

 # Cargar modelo de lenguaje pre-entrenado
generator = pipeline("text-generation", model="gpt2")




Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Device set to use cpu


In [ ]:
# Definir el esquema de solicitud
class TextRequest(BaseModel):
    text: str
    max_length: int = 50

#Rutas de la API
@app.get("/")
def read_root():
  return {"message": "Bienvenido a mi API LLM con Fast API"}

@app.post("/generate")
def generate_text(request: TextRequest):
    try:
        # Generar texto con configuración explícita
        response = generator(
            request.text,
            max_length=request.max_length,
            num_return_sequences=1,
            truncation=True,  # Activar truncamiento explícito
            pad_token_id=50256  # ID de token para padding (GPT-2)
        )
        return {"generated_text": response[0]["generated_text"]}
    except Exception as e:
        # Manejo de errores para evitar fallos en la API
        return {"error": str(e)}

# Exponer la API usando ngrok
public_url = ngrok.connect(8000)
print(f"La API está disponible públicamente en: {public_url}")

# Ejecutar la API
uvicorn.run(app, host="0.0.0.0", port=8000)

La API está disponible públicamente en: NgrokTunnel: "https://9928-34-127-32-130.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [407]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     190.233.58.248:0 - "GET / HTTP/1.1" 200 OK
INFO:     190.233.58.248:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     190.233.58.248:0 - "POST /generate HTTP/1.1" 200 OK
